# Dashboards Avanzados con Plotly y Streamlit

## Objetivos
- Crear dashboards interactivos para análisis de datos
- Utilizar Plotly para visualizaciones dinámicas
- Construir una aplicación web con Streamlit
- Integrar modelos de ML en un dashboard
- Desplegar un dashboard como servicio web

## Contexto
En esta práctica, construiremos un dashboard interactivo para visualizar y analizar datos de scouting de jugadores. Utilizaremos el dataset que hemos trabajado en prácticas anteriores y lo presentaremos de una manera accesible para entrenadores y directivos.

## 1. Preparación del Entorno

### Instalación de Dependencias

In [ ]:
!pip install streamlit pandas plotly scikit-learn joblib

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

## 2. Creación del Dataset

Usaremos un dataset sintético de jugadores con métricas de rendimiento.

In [ ]:
# Generar datos sintéticos de jugadores
np.random.seed(42)
n_players = 500

data = {
    'player_name': [f'Player_{i}' for i in range(n_players)],
    'age': np.random.randint(18, 35, n_players),
    'position': np.random.choice(['GK', 'DEF', 'MID', 'FWD'], n_players, p=[0.1, 0.4, 0.3, 0.2]),
    'market_value': np.random.lognormal(mean=16, sigma=1.5, size=n_players).round(-5),
    'goals': np.random.poisson(5, n_players),
    'assists': np.random.poisson(5, n_players),
    'pass_accuracy': np.random.uniform(0.6, 0.98, n_players),
    'tackles_won': np.random.poisson(30, n_players),
    'dribbles_completed': np.random.poisson(20, n_players),
    'minutes_played': np.random.randint(500, 3000, n_players)
}

df_players = pd.DataFrame(data)

# Ajustar métricas por posición
df_players.loc[df_players['position'] == 'GK', ['goals', 'assists', 'dribbles_completed']] = 0
df_players.loc[df_players['position'] == 'DEF', 'goals'] = (df_players['goals'] * 0.2).round()
df_players.loc[df_players['position'] == 'FWD', 'tackles_won'] = (df_players['tackles_won'] * 0.3).round()

# Guardar dataset
df_players.to_csv('player_scouting_data.csv', index=False)

print("Dataset de scouting creado:")
print(df_players.head())

## 3. Creación de la Aplicación Streamlit

Ahora, crearemos el script de la aplicación Streamlit. Este código debe guardarse en un archivo `.py` (por ejemplo, `dashboard_app.py`) y ejecutarse con `streamlit run dashboard_app.py`.

In [ ]:
streamlit_app_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import numpy as np

# --- Configuración de la Página ---
st.set_page_config(
    page_title="Dashboard de Scouting de Fútbol",
    page_icon="⚽",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Carga de Datos ---
@st.cache_data
def load_data(filepath):
    df = pd.read_csv(filepath)
    return df

df = load_data('player_scouting_data.csv')

# --- Lógica del Buscador de Jugadores Similares ---
def find_similar_players(player_name, df, n_similar=5):
    features = ['age', 'market_value', 'goals', 'assists', 'pass_accuracy', 'tackles_won', 'dribbles_completed', 'minutes_played']
    df_features = df[features]
    
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df_features)
    
    nn = NearestNeighbors(n_neighbors=n_similar + 1)
    nn.fit(df_scaled)
    
    player_idx = df[df['player_name'] == player_name].index[0]
    player_vector = df_scaled[player_idx].reshape(1, -1)
    
    distances, indices = nn.kneighbors(player_vector)
    
    similar_indices = indices.flatten()[1:]
    return df.iloc[similar_indices]

# --- Barra Lateral de Filtros ---
st.sidebar.header("Filtros de Búsqueda")

pos_filter = st.sidebar.multiselect(
    "Posición",
    options=df['position'].unique(),
    default=df['position'].unique()
)

age_filter = st.sidebar.slider(
    "Rango de Edad",
    min_value=int(df['age'].min()),
    max_value=int(df['age'].max()),
    value=(int(df['age'].min()), int(df['age'].max()))
)

value_filter = st.sidebar.slider(
    "Valor de Mercado (millones €)",
    min_value=0.0,
    max_value=float(df['market_value'].max() / 1e6),
    value=(0.0, float(df['market_value'].max() / 1e6))
)

# Aplicar filtros
df_filtered = df[
    (df['position'].isin(pos_filter)) &
    (df['age'].between(age_filter[0], age_filter[1])) &
    (df['market_value'].between(value_filter[0] * 1e6, value_filter[1] * 1e6))
]

# --- Cuerpo Principal del Dashboard ---
st.title("⚽ Dashboard de Scouting de Fútbol")
st.markdown("Análisis interactivo de rendimiento de jugadores.")

# --- Métricas Clave ---
col1, col2, col3, col4 = st.columns(4)
col1.metric("Jugadores Encontrados", f"{df_filtered.shape[0]}")
col2.metric("Valor de Mercado Promedio", f"€{df_filtered['market_value'].mean() / 1e6:.2f}M")
col3.metric("Edad Promedio", f"{df_filtered['age'].mean():.1f} años")
col4.metric("Goles Totales", f"{df_filtered['goals'].sum()}")

st.markdown("---")

# --- Visualizaciones ---
st.header("Análisis Visual")

c1, c2 = st.columns((7, 3))

with c1:
    st.subheader("Distribución de Valor de Mercado vs. Edad")
    fig_scatter = px.scatter(
        df_filtered,
        x="age",
        y="market_value",
        color="position",
        size="goals",
        hover_name="player_name",
        title="Valor de Mercado, Edad y Goles por Posición",
        labels={'age': 'Edad', 'market_value': 'Valor de Mercado (€)', 'position': 'Posición'}
    )
    st.plotly_chart(fig_scatter, use_container_width=True)

with c2:
    st.subheader("Jugadores por Posición")
    fig_pie = px.pie(
        df_filtered,
        names='position',
        title='Distribución de Posiciones'
    )
    st.plotly_chart(fig_pie, use_container_width=True)

# --- Tabla de Datos ---
st.header("Datos de Jugadores")
st.dataframe(df_filtered)

st.markdown("---")

# --- Buscador de Jugadores Similares ---
st.header("Buscador de Jugadores Similares")
player_list = df['player_name'].unique()
selected_player = st.selectbox("Selecciona un jugador para encontrar perfiles similares:", player_list)

if st.button("Buscar Similares"):
    similar_players = find_similar_players(selected_player, df)
    st.subheader(f"Jugadores similares a {selected_player}:")
    st.dataframe(similar_players)
    
    # Gráfico de radar para comparar
    st.subheader("Comparación de Perfiles (Radar Chart)")
    
    player_to_compare = df[df['player_name'] == selected_player]
    comparison_df = pd.concat([player_to_compare, similar_players])
    
    radar_features = ['goals', 'assists', 'pass_accuracy', 'tackles_won', 'dribbles_completed']
    
    fig_radar = go.Figure()
    
    for i, row in comparison_df.iterrows():
        values = row[radar_features].values
        fig_radar.add_trace(go.Scatterpolar(
            r=values,
            theta=radar_features,
            fill='toself',
            name=row['player_name']
        ))
        
    fig_radar.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, max(df[radar_features].max())]
            )
        ),
        showlegend=True
    )
    st.plotly_chart(fig_radar, use_container_width=True)
'''

# Guardar el código de la app Streamlit
with open('dashboard_app.py', 'w') as f:
    f.write(streamlit_app_code)

print("Aplicación Streamlit creada en 'dashboard_app.py'")

## 4. Ejecución de la Aplicación

Para ejecutar el dashboard, abre una terminal y corre el siguiente comando:

```bash
streamlit run dashboard_app.py
```

Esto abrirá una nueva pestaña en tu navegador con el dashboard interactivo.

## 5. Ejercicios Prácticos

### Ejercicio 1: Personalizar el Dashboard
1. **Cambia el tema:** Investiga la documentación de Streamlit y cambia el tema del dashboard (e.g., a un tema oscuro).
2. **Agrega un logo:** Añade un logo de un equipo de fútbol en la barra lateral.
3. **Mejora las métricas:** Incluye métricas adicionales como "Asistencias Totales" o "Promedio de Pases Completados".

### Ejercicio 2: Nuevas Visualizaciones
1. **Histograma de Edades:** Crea un histograma interactivo que muestre la distribución de edades de los jugadores.
2. **Gráfico de Barras:** Añade un gráfico de barras que compare el número de goles promedio por posición.
3. **Mapa de Calor:** Si tuvieras datos de correlación, ¿cómo añadirías un mapa de calor para mostrar la relación entre métricas?

### Ejercicio 3: Integrar un Modelo Predictivo
1. **Cargar el modelo:** Carga el modelo de predicción de valor de mercado que creaste en la práctica anterior (`player_value_model.pkl`).
2. **Crear una nueva sección:** Añade una sección en el dashboard llamada "Predicción de Valor de Mercado".
3. **Formulario de entrada:** Crea un formulario donde un usuario pueda introducir las características de un jugador (edad, goles, etc.).
4. **Mostrar predicción:** Al enviar el formulario, usa el modelo para predecir el valor de mercado y muéstralo en el dashboard.

### Ejercicio 4: Despliegue
1. **Crea un `requirements.txt`:** Asegúrate de tener un archivo con todas las dependencias necesarias.
2. **Despliega en Streamlit Cloud:** Investiga cómo desplegar tu aplicación de forma gratuita en Streamlit Community Cloud.

## Reflexiones y Tareas

1. **¿Qué ventajas ofrece un dashboard interactivo sobre un reporte estático?**
2. **¿Cómo adaptarías este dashboard para diferentes audiencias (entrenador, scout, directivo)?**
3. **¿Qué consideraciones de rendimiento tuviste al trabajar con los datos?**
4. **¿Cómo podrías incorporar datos en tiempo real en este dashboard?**

### Recursos Adicionales

- [Streamlit Documentation](https://docs.streamlit.io/)
- [Plotly Express Documentation](https://plotly.com/python/plotly-express/)
- [Streamlit Gallery](https://streamlit.io/gallery)

---

**¡Excelente trabajo!** Has aprendido a construir y personalizar dashboards interactivos, una habilidad clave para comunicar insights de datos de manera efectiva.